In [ ]:
!pip install -U torch sahi ultralytics
!pip install ipywidgets

In [ ]:
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image, display

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
image = Image.open("/home/dhfmd/Dinglun/0529_small_RGB_V2_V3.tif")
print(image.size)

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path="/home/dhfmd/Dinglun/yolov9/runs/detect/corn30_combine_freeze_0_lr_0.01_batch_16_wd_0.001_dropout_0/weights/best.pt", # any yolov8/yolov9/yolo11/yolo12/rt-detr det model is supported
    confidence_threshold=0.35,
    device="cuda:0", # or 'cuda:0' if GPU is available
)

In [ ]:
# result = get_sliced_prediction(
#     image=image,
#     detection_model=detection_model,
#     slice_height=512,
#     slice_width=512,
#     overlap_height_ratio=0.2,
#     overlap_width_ratio=0.2,
# )

In [ ]:
# result.export_visuals(export_dir="demo_data/", hide_conf=True)

# image_vis = Image.open("demo_data/prediction_visual.png")
# display(image_vis)

In [ ]:
# img = Image.open("demo_data/prediction_visual.png")
# print(img.size)

In [ ]:
# cropped = img.crop((0, 0, 10240, 10240))  # Upper left corner 1024×1024 area
# cropped.show()

### get_prediction per slice

In [ ]:
'''
import os
import pandas as pd
from sahi.utils.file import save_json
from sahi.utils.cv import read_image
from sahi.predict import get_prediction
from PIL import Image
from osgeo import gdal
import numpy as np
import cv2


def pixel2coord(x, y, geotransform):
    xoff, a, b, yoff, d, e = geotransform
    xp = a * x + b * y + a * 0.5 + b * 0.5 + xoff
    yp = d * x + e * y + d * 0.5 + e * 0.5 + yoff
    return xp, yp


def slice_image_pil(image: Image.Image, slice_height: int, slice_width: int,
                    overlap_height_ratio=0.2, overlap_width_ratio=0.2, return_coordinates=False):
    image_width, image_height = image.size
    stride_h = int(slice_height * (1 - overlap_height_ratio))
    stride_w = int(slice_width * (1 - overlap_width_ratio))

    slices = []
    coords = []
    for top in range(0, image_height, stride_h):
        for left in range(0, image_width, stride_w):
            bottom = min(top + slice_height, image_height)
            right = min(left + slice_width, image_width)
            box = (left, top, right, bottom)
            cropped_image = image.crop(box)
            slices.append(cropped_image)
            coords.append(box)
    return (slices, coords) if return_coordinates else slices


def predict_and_save_each_slice(
    tif_path,
    model,
    output_dir,
    slice_height=512,
    slice_width=512,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    confidence_threshold=0.3
):
    os.makedirs(output_dir, exist_ok=True)
    output_csv_dir = os.path.join(output_dir, "csv")
    output_img_dir = os.path.join(output_dir, "slices")
    os.makedirs(output_csv_dir, exist_ok=True)
    os.makedirs(output_img_dir, exist_ok=True)

    # Read image and geo-info
    raster = gdal.Open(tif_path)
    geotransform = raster.GetGeoTransform()
    image = Image.open(tif_path)

    # Slice
    slice_image_list, box_list = slice_image_pil(
        image=image,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=overlap_height_ratio,
        overlap_width_ratio=overlap_width_ratio,
        return_coordinates=True,
    )

    # Predict each slice
    for i, (slice_img, box) in enumerate(zip(slice_image_list, box_list)):
        left, top, right, bottom = box

        result = get_prediction(slice_img, model, shift_amount=[left, top])

        records = []
        for pred in result.object_prediction_list:
            bbox = pred.bbox
            xmin = bbox.minx
            ymin = bbox.miny
            xmax = bbox.maxx
            ymax = bbox.maxy
            conf = pred.score.value

            center_x = (xmin + xmax) / 2
            center_y = (ymin + ymax) / 2

            # Convert center to global pixel coordinates
            global_center_x = center_x + left
            global_center_y = center_y + top

            # Convert to geographic coordinates
            geo_x, geo_y = pixel2coord(global_center_x, global_center_y, geotransform)

            records.append({
                'xmin': xmin, 'ymin': ymin,
                'xmax': xmax, 'ymax': ymax,
                'center_x': center_x, 'center_y': center_y,
                'global_center_x': global_center_x, 'global_center_y': global_center_y,
                'confidence': conf,
                'geo_x': geo_x, 'geo_y': geo_y,
                'slice_index': i
            })

        df = pd.DataFrame(records)
        csv_name = f"slice_{i}.csv"
        df.to_csv(os.path.join(output_csv_dir, csv_name), index=False)

        # Save slice image with boxes (optional visualization)
        vis_img = np.array(slice_img)[:, :, ::-1].copy()  # PIL -> BGR
        for pred in result.object_prediction_list:
            box = pred.bbox.to_voc_bbox()
            cv2.rectangle(vis_img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 1)
        cv2.imwrite(os.path.join(output_img_dir, f"slice_{i}.png"), vis_img)

        print(f"[✓] Saved: slice_{i}.csv and slice_{i}.png")
'''

In [ ]:
# predict_and_save_each_slice(
#     tif_path="/home/dhfmd/Dinglun/0529_small_RGB_V2_V3.tif",
#     model=detection_model,
#     output_dir="my_sahi_results"
# )

In [ ]:
# import pandas as pd
# import glob
# import os

# pixel_dir = "/home/dhfmd/Dinglun/my_sahi_results/csv"
# combined_csv = "/home/dhfmd/Dinglun/my_sahi_results/final_combined_geo_pixel.csv"

# all_slice_csvs = glob.glob(os.path.join(pixel_dir, "slice_*.csv"))

# df_all = []
# for csv_file in all_slice_csvs:
#     if os.path.getsize(csv_file) == 0:
#         continue
#     try:
#         df = pd.read_csv(csv_file)
#         df["slice_name"] = os.path.basename(csv_file).replace(".csv", "")
#         df_all.append(df)
#     except Exception as e:
#         print(f"❌ Error reading {csv_file}: {e}")

# # Merge and save
# combined_df = pd.concat(df_all, ignore_index=True)
# combined_df.to_csv(combined_csv, index=False)
# print(f"✅ All slices merged to: {combined_csv}")

### get_sliced_prediction

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sahi.predict import get_sliced_prediction
from sahi.utils.cv import read_image
from ultralytics import YOLO
from osgeo import gdal
import cv2

def pixel2coord(x, y, geotransform):
    xoff, a, b, yoff, d, e = geotransform
    xp = a * x + b * y + a * 0.5 + b * 0.5 + xoff
    yp = d * x + e * y + d * 0.5 + e * 0.5 + yoff
    return xp, yp

def sahi_pipeline_full_then_slicewise_save(
    tif_path,
    model_path,
    output_dir,
    slice_height=512,
    slice_width=512,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    confidence_threshold=0.3
):
    os.makedirs(output_dir, exist_ok=True)
    output_csv_dir = os.path.join(output_dir, "csv")
    output_img_dir = os.path.join(output_dir, "slices")
    os.makedirs(output_csv_dir, exist_ok=True)
    os.makedirs(output_img_dir, exist_ok=True)

    image = read_image(tif_path)
    pil_image = Image.open(tif_path)
    raster = gdal.Open(tif_path)
    geotransform = raster.GetGeoTransform()

    model = model_path

    result = get_sliced_prediction(
        image=image,
        detection_model=model,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=overlap_height_ratio,
        overlap_width_ratio=overlap_width_ratio,
    )

    image_width, image_height = pil_image.size
    stride_h = int(slice_height * (1 - overlap_height_ratio))
    stride_w = int(slice_width * (1 - overlap_width_ratio))

    slice_record_dict = {}
    for pred in result.object_prediction_list:
        bbox = pred.bbox
        xmin, ymin, xmax, ymax = bbox.minx, bbox.miny, bbox.maxx, bbox.maxy
        conf = pred.score.value
        center_x = (xmin + xmax) / 2
        center_y = (ymin + ymax) / 2
        geo_x, geo_y = pixel2coord(center_x, center_y, geotransform)

        slice_row = int(center_y // stride_h)
        slice_col = int(center_x // stride_w)
        slice_index = slice_row * (image_width // stride_w + 1) + slice_col

        if slice_index not in slice_record_dict:
            slice_record_dict[slice_index] = []

        slice_record_dict[slice_index].append({
            'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax,
            'center_x': center_x, 'center_y': center_y,
            'confidence': conf,
            'geo_x': geo_x, 'geo_y': geo_y,
            'slice_index': slice_index
        })

    for slice_index, record_list in slice_record_dict.items():
        df = pd.DataFrame(record_list)
        csv_name = os.path.join(output_csv_dir, f"slice_{slice_index}.csv")
        df.to_csv(csv_name, index=False)

        top = (slice_index // (image_width // stride_w + 1)) * stride_h
        left = (slice_index % (image_width // stride_w + 1)) * stride_w
        bottom = min(top + slice_height, image_height)
        right = min(left + slice_width, image_width)
        slice_box = (left, top, right, bottom)
        slice_img = pil_image.crop(slice_box)
        vis_img = np.array(slice_img)[:, :, ::-1].copy()

        for row in record_list:
            x1, y1 = int(row["xmin"] - left), int(row["ymin"] - top)
            x2, y2 = int(row["xmax"] - left), int(row["ymax"] - top)
            cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 1)

        slice_img_path = os.path.join(output_img_dir, f"slice_{slice_index}.png")
        cv2.imwrite(slice_img_path, vis_img)

        print(f"[✓] Saved: slice_{slice_index}.csv and .png")

    return output_csv_dir

In [ ]:
sahi_pipeline_full_then_slicewise_save(
    tif_path="/home/dhfmd/Dinglun/0529_small_RGB_V2_V3.tif",
    model_path=detection_model,
    output_dir="my_sahi_results"
)

In [ ]:
import pandas as pd
import glob
import os

pixel_dir = "/home/dhfmd/Dinglun/my_sahi_results/csv"
combined_csv = "/home/dhfmd/Dinglun/my_sahi_results/final_combined_geo_pixel.csv"

all_slice_csvs = glob.glob(os.path.join(pixel_dir, "slice_*.csv"))

df_all = []
for csv_file in all_slice_csvs:
    if os.path.getsize(csv_file) == 0:
        continue
    try:
        df = pd.read_csv(csv_file)
        df["slice_name"] = os.path.basename(csv_file).replace(".csv", "")
        df_all.append(df)
    except Exception as e:
        print(f"❌ Error reading {csv_file}: {e}")

# Merge and save
combined_df = pd.concat(df_all, ignore_index=True)
combined_df.to_csv(combined_csv, index=False)
print(f"✅ All slices merged to: {combined_csv}")